In [ ]:
%pip install pymorphy2
%pip install examples/datasphere/stt_metrics-0.12-py3-none-any.whl

# Обучение языковых моделей для STT

## Пошаговый гайд

1. Подготовьте данные, необходимые для обучения языковой модели. Для этого требуется подготовить тексты, а также наборы из размеченных валидационных и тестовых аудиозаписей
 
  * Набор текстов, необходимый для обучения языковой модели, должен представлять из себя таблицу в формате TSV, состоящую из двух столбцов, где второй столбец соответствует самим текстам (предложениям), а первый столбец &mdash; количеству вхождений этих текстов в исходный датасет. Также к текстам разметки предъявляются следующие требования:
 
    * все тексты должны быть приведены к нижнему регистру
    
    * тексты не должны содержать символов, отличных от символов русского алфавита и пробелов
    
    * тексты не должны содержать букв "ё", все такие буквы следует заменять на "е"
    
    * тексты не должны содержать пробелов в начале и в конце, также все слова должны быть разделены одним пробелом
    
    Следует отметить, что аналогичные требования также предъявляются и к разметке аудиозаписей
 
  * Валидационные аудиозаписи и их разметка должны быть сохранены в отдельной папке. Эти данные будут использоваться для подбора гиперпараметров обучаемой модели.
 
    Все аудиозаписи должны иметь следующий формат:
    
    * одноканальное аудио в формате WAV
    
    * sample rate: 8000, 16000 или 48000
    
    * разрядность: 16 бит, little endian
 
    Кроме того, вместе с валидационными аудиозаписями должен храниться файл `records.json` с разметкой аудио. Этот файл должен иметь следующий формат:
    
    ```
    {
        "<audio1_name>.wav": "разметка 1-ой записи",
        ...
        "<audioN_name>.wav": "разметка N-ой записи",
    }
    ```
    &nbsp;
  * Тестовые аудиозаписи и их разметка должны быть подготовлены аналогично набору валидационных записей. Эти данные не будут использоваться при обучении, но по ним будет получены распознавания с помощью обученной языковой модели, которые затем могут быть использованы для оценки её качества

2. Запустите команду
   
  ```
    > #!nirvana
    > sk_train_language_model --train-texts PATH --validation-dir PATH --test-dir PATH --model PATH --recognitions PATH
    >
  ```
  * `--train-texts PATH` &mdash; путь до TSV-таблицы с текстами
  
  * `--validation-dir PATH` &mdash; путь до директории с валидационными аудиозаписями
  
  * `--test-dir PATH` &mdash; путь до директории с тестовыми аудиозаписями
  
  * `--model PATH` &mdash; путь, по которому будет сохранена полученная языковая модель
  
  * `--recognitions PATH` &mdash; путь, по которому будет сохранены распознавания аудиозаписей из тестового набора (в формате JSON)

3. Дождитесь завершения команды. На этом этапе вы также можете закрыть вкладку с ноутбуком и вернуться за результатам позже. Обучение модели отработает в фоновом режиме.

   Время обучения напрямую зависит от количества предоставленных текстов, а также от количества аудиозаписей. 
   
   Количество данных, требуемое для обучения хорошей языковой модели, может сильно варьироваться в зависимости от конкретной задачи. Тем не менее, _рекомендуется_ использовать не менее 1МБ текстов, а также от 100 до 2000 валидационных и тестовых аудиозаписей.

## Подготовка данных

Приведём пример того, как подготовить данные для обучения кастомной языковой модели.

Предположим, что изначально в нашем распоряжении имеется набор текстов `raw_data/dataset.txt`, набор аудиозаписей, хранящийся в папке `raw_data/audio`, а также разметка этих аудиозаписей, хранящаяся в папке `raw_data/references`.

### Подготовка текстов

В первую очередь подготовим тексты для обучения языковой модели. Для этого разобьём тексты нашего датасета на предложения, нормализуем их и сохраним в файл `prepared_data/texts.tsv` в соответствии с описанным выше форматом. 

Ниже приведён возможный пример такой обработки текстов:

In [ ]:
import nltk
nltk.download('punkt')

import re
from pathlib import Path
from nltk.tokenize import sent_tokenize
from collections import defaultdict


def normalize_sentence(text: str) -> str:
    # приводим текст предложения к нижнему регистру, заменяем буквы "ё" на "е"
    text = text.lower().replace('ё', 'е')
    
    # удаляем из текста спец. символы, оставляем только русскоязычные символы и пробелы
    text = re.sub(r'[^а-я ]', ' ', text)
    
    # также удаляем все лишние пробелы
    text = ' '.join(filter(len, text.split()))
    
    return text


def prepare_texts(src_path: Path, dst_path: Path, merge_duplicates: bool = True):
    sentences = []
    
    text = src_path.read_text()
    
    # разбиваем текст на предложения и нормализуем их
    for paragraph in text.splitlines():
        for sentence in sent_tokenize(paragraph):
            sentences.append(normalize_sentence(sentence))

    if not dst_path.parent.exists():
        dst_path.parent.mkdir(parents=True)
        
    with dst_path.open('w') as f:
        if not merge_duplicates:
            # в простом варианте сохраняем все предложения отдельно, допуская дубликаты
            for sentence in sentences:
                f.write(f'1\t{sentence}\n')
        else:
            # опционально можем избавиться от дубликатов, сохранив с каждым предложением количество его вхождений в текст
            sentence_count = defaultdict(int)
            for sentence in sentences:
                sentence_count[sentence] += 1
            for sentence, count in sentence_count.items():
                f.write(f'{count}\t{sentence}\n')

In [ ]:
prepare_texts(
    src_path=Path('examples', 'datasphere', 'fit_lm', 'raw_data', 'dataset.txt'), 
    dst_path=Path('examples', 'datasphere', 'fit_lm', 'prepared_data', 'texts.tsv')
)

### Подготовка аудио

Далее подготовим аудиозаписи. 

Для этого разобъём весь набор аудиозаписей на валидационный и тестовый наборы, а затем переведём аудиозаписи и их разметку в требуемый нам формат, разместив их в папках `prepared_data/val_audio` и `prepared_data/test_audio` соответственно.

In [ ]:
%pip install pydub

import random
import json
from pydub import AudioSegment
from typing import List


def prepare_audio_set(src_audio_dir: Path, reference_paths: List[Path], dst_dir: Path):
    if not dst_dir.exists():
        dst_dir.mkdir(parents=True)
    
    records_desc = {}
    
    for reference_path in reference_paths:
        # по имени файла с разметкой получаем путь до соответствующей аудиозаписи
        audio_name = reference_path.name.replace('.txt', '.wav')
        
        src_audio_path = src_audio_dir / audio_name
        dst_audio_path = dst_dir / audio_name
        
        with src_audio_path.open('rb') as f:
            audio = AudioSegment.from_wav(f)
            
        # приводим все аудиозаписи к единому формату, с которым работают наши инструменты
        audio = audio.set_channels(1).set_sample_width(2).set_frame_rate(16000)
        
        # и сохраняем в формате WAV
        with dst_audio_path.open('wb') as f:
            audio.export(out_f=f, format='wav')
        
        # кроме того, нормализуем текст разметки
        records_desc[audio_name] = normalize_sentence(reference_path.read_text())
    
    with (dst_dir / 'records.json').open('w') as f:
        json.dump(records_desc, f, indent=2)


# здесь получим список всех записей (их разметок) и поровну разделяем его на валидационный и тестовый наборы
def prepare_audio(src_audio_dir: Path, src_reference_dir: Path, dst_dir: Path):
    random.seed(0)
    
    reference_paths = list(filter(lambda f: f.name.endswith('.txt'), src_reference_dir.iterdir()))
    random.shuffle(reference_paths)
    
    split_size = len(reference_paths) // 2
    
    prepare_audio_set(src_audio_dir, reference_paths[:split_size], dst_dir / 'val_audio')
    prepare_audio_set(src_audio_dir, reference_paths[split_size:], dst_dir / 'test_audio')

In [ ]:
prepare_audio(
    Path('examples', 'datasphere', 'fit_lm', 'raw_data', 'audio'), 
    Path('examples', 'datasphere', 'fit_lm', 'raw_data', 'references'), 
    Path('examples', 'datasphere', 'fit_lm', 'prepared_data')
)

## Обучение модели

Теперь мы можем вызвать команду `sk_train_language_model`, указав в качестве входных параметров пути до текстов и наборов аудиозаписей.

По завершению исполнения ячейки в файл `lm` будет сохранена языковая модель, а в файл `recognitions.json` будет сохранены результаты потокового распознавания записей из тестового набора в зависимости от некоторых настроек распознавания.

In [ ]:
#!nirvana
sk_train_language_model --train-texts ./examples/datasphere/fit_lm/prepared_data/texts.tsv --validation-dir ./examples/datasphere/fit_lm/prepared_data/val_audio --test-dir ./examples/datasphere/fit_lm/prepared_data/test_audio --model ./examples/datasphere/fit_lm/lm --recognitions ./examples/datasphere/fit_lm/recognitions.json

## Оценка качества модели

Наконец, мы можем оценить качество полученной модели на тестовой выборке, чтобы затем иметь возможность выбрать лучшую модель.

Подробнее о том, как "правильно" оценить качество модели, рассказано [тут](?). Здесь же мы приведём самый простой вариант оценки качества модели в зависимости от использованных параметров распознавания с помощью метрики WER.

In [ ]:
import json
from pathlib import Path

from stt_metrics import evaluate_wer

references = json.loads(Path('examples', 'datasphere', 'fit_lm', 'prepared_data', 'test_audio', 'records.json').read_text())
recognition_sets = json.loads(Path('examples', 'datasphere', 'fit_lm', 'recognitions.json').read_text())

for recognition_set in recognition_sets:
    mean_wer, full_report = evaluate_wer(references=references, hypotheses=recognition_set['recognitions'])
    print(f'Recognition params: {recognition_set["params"]}')
    print(f'Mean WER: {mean_wer}\n')